<a href="https://colab.research.google.com/github/heesukjang/W209_DataViz_Summer2023/blob/main/Unit_Assignments/Jesse_Week_08_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nx_altair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import json
import nx_altair as nxa
import networkx as nx
from google.colab import drive
drive.mount('/content/drive')

pd.__version__

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'1.5.3'

You will likely have to upload the file `players_20.csv` to the workspace yourself.

In [ ]:
filename = "/content/drive/MyDrive/duto_guerra_tweets.json"

with open(filename, 'r') as f:
    content = f.read()

d = json.loads(content)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean_hashtags(hashtag: dict) -> list:
    return hashtag['text']

def clean_json(tweet: dict) -> list:
    hashtags = tweet['entities']['hashtags']

    if len(hashtags)>0:
        hashtags = list(map(clean_hashtags, hashtags))

    return hashtags


In [ ]:
hashtags = list(filter(lambda x: len(x)>0, map(clean_json, d)))


graph = nx.Graph()
for tweet_hashtags in hashtags:
    for i in range(len(tweet_hashtags)):
        for j in range(i+1, len(tweet_hashtags)):
            if graph.get_edge_data(tweet_hashtags[i], tweet_hashtags[j]) is None:
                graph.add_edge(tweet_hashtags[i], tweet_hashtags[j], weight=1)
            else:
                graph[tweet_hashtags[i]][tweet_hashtags[j]]['weight'] += 1


# for hashtag in hashtags:
#     graph.add_node(hashtag, hashtag=hashtag)

In [ ]:
pos = nx.spring_layout(graph)

In [ ]:
chart = nxa.draw_networkx(
    graph, pos=pos,
    node_color='skyblue',
    edge_color='gray',
    node_size=20,
    # node_size = 'weight',
)

labels_df = pd.DataFrame(pos).T.reset_index()
labels_df.columns = ['id', 'x', 'y']
labels = alt.Chart(labels_df).mark_text(
    align='left',
    dx=3, dy=3
).encode(
    x='x:Q',
    y='y:Q',
    text='id:N'
)

# Overlay labels on the original chart
chart = chart + labels

chart = chart.properties(
    width=1600,
    height=800,
    title="Network graph of tweets"
)
# Display the chart
chart.interactive()


alt.LayerChart(...)